In [52]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
API_KEY = "..." #Colocar API aqui, deixarei em branco para não ser utilizado minhas cotas

In [53]:
df = pd.read_csv('data.csv')

In [ ]:
#Utilizei apenas as 100 primeiras linhas por conta do tempo de processamento e custo de chamada de API
#Apenas quis dar uma noção de uma abordagem mais complexa e provavelmente mais eficiente.
df = df.head(100)

In [64]:
llm = ChatOpenAI(openai_api_key=API_KEY, model_name="gpt-3.5-turbo")

In [65]:
#Setando prompt, defino para a GenAI retornar True or False pro problema.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are the responsible to detect companies for water treatment."),
    ("user", "Based on the follow description of the company, return True or False if it is a water treatment company: {description}. Do not return any other response than True or False.")
])

In [66]:
#Loop chamando o prompt com a description de cada uma das rows:
def call_chat_gpt(description):
    llmchain = LLMChain(llm=llm, prompt=prompt)
    response = llmchain.run(description)
    if "True" in response:
        return True
    else:
        return False

df["is_water_company"] = df.apply(lambda row: call_chat_gpt(row["description"]) , axis=1)

In [70]:
results = df

In [71]:
results

,name,description,employees,total_funding,city,subcountry,lat,lng,is_water_company
0,fitin,fitin is the latest fit tech platform that pro...,1.0,NaN,toronto,ontario,43.700109999999995,-79.4163,False
1,quinditech,our services include independent & collaborati...,10.0,NaN,montreal,quebec,45.50884,-73.58781,False
2,botfirst,chatlanding is a digital chatbot platform wher...,1.0,NaN,toronto,ontario,43.700109999999995,-79.4163,False
3,bravver,ai-powered army of health professionals in you...,1.0,NaN,quebec,quebec,46.8259601,-71.23522259999999,False
4,stylify,stylify is a mobile platform that connects peo...,1.0,NaN,toronto,ontario,43.700109999999995,-79.4163,False
...,...,...,...,...,...,...,...,...,...
95,swirltex,swirltex has developed a membrane filtration s...,1.0,NaN,calgary,alberta,51.05011,-114.08528999999999,True
96,excilia,excilia tracks employee mental health and prov...,1.0,NaN,waterloo,ontario,43.466874,-80.524635,False
97,flourish,flourish is a protein pancake mix that blend t...,1.0,NaN,vaughan,ontario,43.8361,-79.49826999999998,False
98,specialvr,we create multi-sensory environments in virtua...,1.0,NaN,waterloo,ontario,43.466874,-80.524635,False


In [73]:
water_company = results[results["is_water_company"] == True]

In [75]:
for index, row in water_company.iterrows():
    print("Company Name:", row['name'])
    print("Description:", row['description'])
    print("-----------------------------------")

Company Name: swirltex 
Description: swirltex has developed a membrane filtration system for treating wastewater.the swirltex lagoon unit (slu) has potential to be transformative for municipalities reliant on lagoon systems as well as for industrial applications. wastewater has traditionally been difficult to treat through membrane filtration due to surface fouling by pollutants. the slu overcomes fouling by creating a centrifugal flow pattern, preventing contaminants from contacting the tubular membrane, thus creating a much more effective system. swirltex will sell the service of treated water per unit volume. it is an affordable option for smaller centers that cannot afford large capital outlays and creates marketable reuse water. the initial target market is municipalities at risk of environmental non-compliance. for the future, swirltex will be developing oil and gas applications. our goal is to supply the market demand for recycled industrial water, which will continue to grow wi

In [76]:
water_company.to_csv("water_companies_by_chat_gpt.csv", index=False)